<a href="https://colab.research.google.com/github/VittorioBartolomeoSecondin/DVIS-CAValli_Team/blob/main/Temperatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data exploration and preprocessing

## Importing libraries and connecting to Google Drive

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import re
import json
import csv
import requests
import os
import zipfile
from google.colab import files
from google.colab import drive
drive.mount('/content/drive') # turned out to be a good workaround to load a huge amount of data and keep it available

Mounted at /content/drive


In [40]:
tree_dataset = pd.read_csv("/content/drive/MyDrive/tree_dataset/final_dataset/tree_dataset.csv", low_memory=False)

In [84]:
all_files = glob.glob("/content/drive/MyDrive/tree_dataset/final_dataset/*.txt")
all_files.sort()
print(all_files)

['/content/drive/MyDrive/tree_dataset/final_dataset/climdiv-tmaxst-v1.0.0-20231106.txt', '/content/drive/MyDrive/tree_dataset/final_dataset/climdiv-tminst-v1.0.0-20231106.txt', '/content/drive/MyDrive/tree_dataset/final_dataset/climdiv-tmpcst-v1.0.0-20231106.txt']


In [85]:
column_names = ['Code', 'JanF', 'FebF', 'MarF', 'AprF', 'MayF', 'JunF', 'JulF', 'AugF', 'SepF', 'OctF', 'NovF', 'DecF']
column_data_types = {column_names[0]: str}
column_data_types.update({column_names[i]: float for i in range(1, 13)})
max_dataset = pd.read_csv(all_files[0], delimiter = r'\s+', header = None, names = column_names, dtype = column_data_types)
min_dataset = pd.read_csv(all_files[1], delimiter = r'\s+', header = None, names = column_names, dtype = column_data_types)
avg_dataset = pd.read_csv(all_files[2], delimiter = r'\s+', header = None, names = column_names, dtype = column_data_types)

In [86]:
max_dataset.replace(-99.9, np.nan, inplace=True)
min_dataset.replace(-99.9, np.nan, inplace=True)
avg_dataset.replace(-99.9, np.nan, inplace=True)

In [87]:
months_F = column_names[1:]
months_C = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for idx, month in enumerate(months_C):
  max_dataset[month] = ((max_dataset[months_F[idx]] - 32) * 5 / 9).round(1)
  min_dataset[month] = ((min_dataset[months_F[idx]] - 32) * 5 / 9).round(1)
  avg_dataset[month] = ((avg_dataset[months_F[idx]] - 32) * 5 / 9).round(1)

In [88]:
avg_dataset['Code']

0        0010021895
1        0010021896
2        0010021897
3        0010021898
4        0010021899
            ...    
12478    3650022019
12479    3650022020
12480    3650022021
12481    3650022022
12482    3650022023
Name: Code, Length: 12483, dtype: object

In [ ]:
STATE-CODE        1-3    STATE-CODE as indicated in State Code Table above.
                         Range of values is 001-110 for standard states,
                         regions and national,  111-465 for special regions.

DIVISION-NUMBER    4     DIVISION NUMBER.  Value is 0 which indicates an area-
                         averaged element.

ELEMENT-CODE      5-6    02 = Average Temperature
                         27 = Maximum Temperature
                         28 = Minimum Temperature

YEAR

In [89]:


max_dataset['state_code'] = max_dataset['Code'][:3]
max_dataset['division_number'] = max_dataset['Code'][3]
max_dataset['element_code'] = max_dataset['Code'][4:6]
max_dataset['year'] = max_dataset['Code'][6:]

,Code,JanF,FebF,MarF,AprF,MayF,JunF,JulF,AugF,SepF,...,Jul,Aug,Sep,Oct,Nov,Dec,state_code,division_number,element_code,year
0,0010271895,52.7,48.1,66.5,75.7,80.6,88.4,89.6,89.7,89.1,...,32.0,32.1,31.7,23.4,18.4,13.9,0010271895,0010271898,NaN,NaN
1,0010271896,53.0,59.0,63.9,80.6,87.8,87.9,91.6,94.0,89.2,...,33.1,34.4,31.8,24.3,20.5,14.3,0010271896,0010271898,NaN,NaN
2,0010271897,52.1,61.0,69.9,74.8,82.2,94.0,92.8,89.5,88.7,...,33.8,31.9,31.5,27.2,19.5,14.4,0010271897,0010271898,NaN,NaN
3,0010271898,59.1,58.0,69.9,71.0,88.0,92.7,90.5,88.4,85.4,...,32.5,31.3,29.7,22.4,16.1,12.4,NaN,0010271898,NaN,NaN
4,0010271899,54.4,51.4,67.8,72.6,88.6,92.3,91.9,92.0,86.3,...,33.3,33.3,30.2,25.6,19.9,13.1,NaN,0010271898,0010271899,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12478,3650272019,55.5,59.6,63.7,74.8,81.0,88.7,92.4,93.6,90.8,...,33.6,34.2,32.7,24.4,17.2,15.0,NaN,0010271898,NaN,3650272019
12479,3650272020,58.1,59.1,68.8,73.5,82.5,89.5,93.9,92.9,85.0,...,34.4,33.8,29.4,25.1,20.7,14.2,NaN,0010271898,NaN,3650272020
12480,3650272021,54.9,53.3,68.4,73.6,80.6,89.4,90.9,91.4,87.6,...,32.7,33.0,30.9,25.7,19.1,18.4,NaN,0010271898,NaN,3650272021
12481,3650272022,55.7,59.0,68.8,76.0,85.1,92.0,95.1,91.2,88.0,...,35.1,32.9,31.1,24.9,17.3,13.5,NaN,0010271898,NaN,3650272022


In [ ]:
001 Alabama         030 New York
002 Arizona         031 North Carolina
003 Arkansas        032 North Dakota
004 California      033 Ohio
005 Colorado        034 Oklahoma
006 Connecticut     035 Oregon
007 Delaware        036 Pennsylvania
008 Florida         037 Rhode Island
009 Georgia         038 South Carolina
010 Idaho           039 South Dakota
011 Illinois        040 Tennessee
012 Indiana         041 Texas
013 Iowa            042 Utah
014 Kansas          043 Vermont
015 Kentucky        044 Virginia
016 Louisiana       045 Washington
017 Maine           046 West Virginia
018 Maryland        047 Wisconsin
019 Massachusetts   048 Wyoming
020 Michigan        050 Alaska
021 Minnesota       101 Northeast Region
022 Mississippi     102 East North Central Region
023 Missouri        103 Central Region
024 Montana         104 Southeast Region
025 Nebraska        105 West North Central Region
026 Nevada          106 South Region
027 New Hampshire   107 Southwest Region
028 New Jersey      108 Northwest Region
029 New Mexico      109 West Region
                    110 National (contiguous 48 States)
111 Great Plains
115 Southern Plains and Gulf Coast
120 US Rockies and Westward
121 NWS Eastern Region
122 NWS Southern Region
123 NWS Central Region
124 NWS Western Region
201 Pacific Northwest Basin
202 California River Basin
203 Great Basin
204 Lower Colorado River Basin
205 Upper Colorado River Basin
206 Rio Grande River Basin
207 Texas Gulf Coast River Basin
208 Arkansas-White-Red Basin
209 Lower Mississippi River Basin
210 Missouri River Basin
211 Souris-Red-Rainy Basin
212 Upper Mississippi River Basin
213 Great Lakes Basin
214 Tennessee River Basin
215 Ohio River Basin
216 South Atlantic-Gulf Basin
217 Mid-Atlantic Basin
218 New England Basin
220 Mississippi River Basin & Tributaties (N. of Memphis, TN)

250 Spring Wheat Belt (area weighted)
255 Primary Hard Red Winter Wheat Belt (area weighted)
256 Winter Wheat Belt (area weighted)
260 Primary Corn and Soybean Belt (area weighted)
261 Corn Belt (area weighted)
262 Soybean Belt (area weighted)
265 Cotton Belt (area weighted)

350 Spring Wheat Belt (productivity weighted)
356 Winter Wheat Belt (productivity weighted)
361 Corn Belt (productivity weighted)
362 Soybean Belt (productivity weighted)
365 Cotton Belt (productivity weighted)

450 Spring Wheat Belt (% productivity in the Palmer Z Index)
456 Winter Wheat Belt (% productivity in the Palmer Z Index)
461 Corn Belt (% productivity in the Palmer Z Index)
462 Soybean Belt (% productivity in the Palmer Z Index)
465 Cotton Belt (% productivity in the Palmer Z Index)

STATE-CODE        1-3    STATE-CODE as indicated in State Code Table above.
                         Range of values is 001-110 for standard states,
                         regions and national,  111-465 for special regions.

DIVISION-NUMBER    4     DIVISION NUMBER.  Value is 0 which indicates an area-
                         averaged element.

ELEMENT-CODE      5-6    02 = Average Temperature
                         27 = Maximum Temperature
                         28 = Minimum Temperature

YEAR              7-10   This is the year of record.  Range is 1895 to current year processed.

In [ ]:
001 Alabama         s030 New York
s002 Arizona         s031 North Carolina
003 Arkansas        032 North Dakota
s004 California      s033 Ohio
s005 Colorado        s034 Oklahoma
006 Connecticut     s035 Oregon
s007 Delaware        s036 Pennsylvania
s008 Florida         s037 Rhode Island
s009 Georgia         038 South Carolina
010 Idaho           s039 South Dakota
011 Illinois        s040 Tennessee
s012 Indiana         s041 Texas
s013 Iowa            042 Utah
s014 Kansas          043 Vermont
s015 Kentucky        s044 Virginia
s016 Louisiana       s045 Washington
017 Maine           046 West Virginia
s018 Maryland        s047 Wisconsin
s019 Massachusetts   048 Wyoming
s020 Michigan        050 Alaska
s021 Minnesota       101 Northeast Region
022 Mississippi     102 East North Central Region
s023 Missouri        103 Central Region
024 Montana         104 Southeast Region
025 Nebraska        105 West North Central Region
s026 Nevada          106 South Region
027 New Hampshire   107 Southwest Region
s028 New Jersey      108 Northwest Region
s029 New Mexico      109 West Region
                              110 National (contiguous 48 States)


# Download ALL .csv files

In [ ]:
directory_path = '/content'
file_type = ".csv"
files_to_download = glob.glob(f"{directory_path}/*{file_type}")
zip_filename = "datasets.zip"
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for file in files_to_download:
        zipf.write(file, arcname=os.path.basename(file))
files.download(zip_filename)